In [39]:
import pandas as pd
import pickle
import os.path 
import re
import spacy
import Levenshtein

In [40]:
def read_pickle(texts: str):
    # Открытие файла texts.pickle для чтения в бинарном режиме
    with open(os.path.join(texts), 'rb') as file:
        # Загрузка данных из файла и возвращение их
        return pickle.load(file)

# Преобразование загруженных данных в DataFrame
dfmentions = pd.DataFrame(read_pickle('mentions_texts.pickle'))
dfsetiments = pd.DataFrame(read_pickle('sentiment_texts.pickle'))

In [41]:
dfsetiments

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [42]:
name_company = pd.read_excel('issuerid.xlsx')

In [43]:
dict_company_names = {row[0]: [val for val in row[1:] if pd.notna(val)] for row in name_company.values.tolist()}
dict_company_names

{1: ['Акционерный коммерческий банк "Держава" публичное акционерное общество',
  'DERZP',
  'Держава',
  'DERZHAVA',
  'DERZ'],
 2: ['"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
  'CBOM RX',
  'Московский кредитный банк',
  'мкб',
  'Credit Bank of Moscow',
  'Credit Bank'],
 3: ['"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
  'RDRB',
  'Российский акционерный коммерческий дорожный банк',
  'РДБанк',
  'Дорожный банк',
  'Russian public joint-stock commercial roads Bank',
  'RosDorBank',
  'РосДорБанк',
  'roads Bank'],
 4: ['Акционерная компания "АЛРОСА" (публичное акционерное общество)',
  'ALRS RX',
  'алроса',
  'alrosa'],
 5: ['Акционерный Коммерческий банк "АВАНГАРД" - публичное акционерное общество',
  'AVAN',
  'Авангард',
  'AVANGARD'],
 6: ['акционерный коммерческий банк "Приморье"  (публичное акционерное общество)',
  'PRMB',
  'АКБ Приморье',
  'АКБ "Приморье"',
  'банк "Приморье"',
  'банк Приморье',
  'primbank'

In [44]:
company_names = sum([[val for i, val in enumerate(row) if pd.notna(val) and i!=0] for row in name_company.values.tolist()], [])
company_names

['Акционерный коммерческий банк "Держава" публичное акционерное общество',
 'DERZP',
 'Держава',
 'DERZHAVA',
 'DERZ',
 '"МОСКОВСКИЙ КРЕДИТНЫЙ БАНК" (публичное акционерное общество)',
 'CBOM RX',
 'Московский кредитный банк',
 'мкб',
 'Credit Bank of Moscow',
 'Credit Bank',
 '"Российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
 'RDRB',
 'Российский акционерный коммерческий дорожный банк',
 'РДБанк',
 'Дорожный банк',
 'Russian public joint-stock commercial roads Bank',
 'RosDorBank',
 'РосДорБанк',
 'roads Bank',
 'Акционерная компания "АЛРОСА" (публичное акционерное общество)',
 'ALRS RX',
 'алроса',
 'alrosa',
 'Акционерный Коммерческий банк "АВАНГАРД" - публичное акционерное общество',
 'AVAN',
 'Авангард',
 'AVANGARD',
 'акционерный коммерческий банк "Приморье"  (публичное акционерное общество)',
 'PRMB',
 'АКБ Приморье',
 'АКБ "Приморье"',
 'банк "Приморье"',
 'банк Приморье',
 'primbank',
 'PJSCB "Primorye"',
 'PJSCB Primorye',
 'public joint-s

In [45]:
import nltk
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Глеб\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [46]:
company_names_lower = [company_name.lower() for company_name in company_names]
company_names_lower

['акционерный коммерческий банк "держава" публичное акционерное общество',
 'derzp',
 'держава',
 'derzhava',
 'derz',
 '"московский кредитный банк" (публичное акционерное общество)',
 'cbom rx',
 'московский кредитный банк',
 'мкб',
 'credit bank of moscow',
 'credit bank',
 '"российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
 'rdrb',
 'российский акционерный коммерческий дорожный банк',
 'рдбанк',
 'дорожный банк',
 'russian public joint-stock commercial roads bank',
 'rosdorbank',
 'росдорбанк',
 'roads bank',
 'акционерная компания "алроса" (публичное акционерное общество)',
 'alrs rx',
 'алроса',
 'alrosa',
 'акционерный коммерческий банк "авангард" - публичное акционерное общество',
 'avan',
 'авангард',
 'avangard',
 'акционерный коммерческий банк "приморье"  (публичное акционерное общество)',
 'prmb',
 'акб приморье',
 'акб "приморье"',
 'банк "приморье"',
 'банк приморье',
 'primbank',
 'pjscb "primorye"',
 'pjscb primorye',
 'public joint-s

In [47]:
dfsetiments

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [48]:
messages_and_issuerid = dfsetiments.groupby('MessageText')['issuerid'].apply(list).reset_index()
messages_and_issuerid

,MessageText,issuerid
0,,[235]
1,""" США вводят санкции против российских банков:...",[100]
2,"""""""АКРОН"""" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...",[24]
3,"""""""АЛРОСА"""" может выплатить дивиденды по итога...",[4]
4,"""""""Алекперов один был готов заплатить за долю ...",[236]
...,...,...
7183,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,[99]
7184,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,[99]
7185,🪨 Мечел: акции с потенциалом роста свыше 90% д...,[99]
7186,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,[99]


In [49]:
messages_and_SentimentScore = dfsetiments.groupby('MessageText')['SentimentScore'].apply(list).reset_index()
messages_and_SentimentScore

,MessageText,SentimentScore
0,,[4]
1,""" США вводят санкции против российских банков:...",[2]
2,"""""""АКРОН"""" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...",[2]
3,"""""""АЛРОСА"""" может выплатить дивиденды по итога...",[4]
4,"""""""Алекперов один был готов заплатить за долю ...",[4]
...,...,...
7183,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,[4]
7184,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,[5]
7185,🪨 Мечел: акции с потенциалом роста свыше 90% д...,[4]
7186,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,[4]


In [50]:
messages_sentiments_company = messages_and_issuerid.copy()
messages_sentiments_company['SentimentScores'] = dfsetiments.groupby('MessageText')['SentimentScore'].apply(list).reset_index()['SentimentScore']
messages_sentiments_company

,MessageText,issuerid,SentimentScores
0,,[235],[4]
1,""" США вводят санкции против российских банков:...",[100],[2]
2,"""""""АКРОН"""" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...",[24],[2]
3,"""""""АЛРОСА"""" может выплатить дивиденды по итога...",[4],[4]
4,"""""""Алекперов один был готов заплатить за долю ...",[236],[4]
...,...,...,...
7183,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,[99],[4]
7184,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,[99],[5]
7185,🪨 Мечел: акции с потенциалом роста свыше 90% д...,[99],[4]
7186,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,[99],[4]


In [51]:
messages_sentiments_company['n_companies_in_message'] = messages_sentiments_company['issuerid'].apply(list).reset_index()['issuerid'].apply(len)

In [52]:
messages_sentiments_company

,MessageText,issuerid,SentimentScores,n_companies_in_message
0,,[235],[4],1
1,""" США вводят санкции против российских банков:...",[100],[2],1
2,"""""""АКРОН"""" БУДЕТ ОСПАРИВАТЬ РЕШЕНИЕ ПОЛЬШИ ПО ...",[24],[2],1
3,"""""""АЛРОСА"""" может выплатить дивиденды по итога...",[4],[4],1
4,"""""""Алекперов один был готов заплатить за долю ...",[236],[4],1
...,...,...,...,...
7183,🪨 Взгляд на компанию: «Мечел» — эффект от отме...,[99],[4],1
7184,🪨 Взгляд на компанию: «Мечел»: 3-й кв. 2023 г....,[99],[5],1
7185,🪨 Мечел: акции с потенциалом роста свыше 90% д...,[99],[4],1
7186,🪨 Мечел: анализ ключевых тем. Взгляд БКС Мы п...,[99],[4],1


In [53]:
messages_sentiments_company.sort_values(by='n_companies_in_message')

,MessageText,issuerid,SentimentScores,n_companies_in_message
0,,[235],[4],1
4662,🇷🇺#ABIO После интервью Марии Воронцовой о раз...,[266],[4],1
4661,🇷🇺#ABIO «Артген биотех» отчитался за девять м...,[266],[3],1
4660,🇷🇺#5G #связь #телекомы #россия МТС приступила...,[100],[4],1
4658,🇷🇺 Российский рынок акций в среду снова прибав...,[225],[4],1
...,...,...,...,...
5753,💡 Подборка лучших акций российских компаний дл...,"[115, 127, 127, 112, 47, 160, 150, 36, 152, 11...","[4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 2, 4, 4]",13
2559,Какие интриги еще остались в дивидендном сезон...,"[235, 228, 150, 225, 89, 36, 90, 163, 103, 99,...","[3, 3, 3, 3, 2, 3, 2, 4, 3, 2, 4, 4, 3]",13
522,"""✍️ Про рынок: Какие акции выбирают телеграм-к...","[111, 223, 7, 89, 236, 112, 237, 254, 116, 160...","[4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 5, 4, 4, 4, 4]",16
3323,Топ-30 российских компаний по капитализации. П...,"[111, 230, 11, 47, 160, 163, 4, 152, 126, 185,...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...",17


In [54]:
print(messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[-1]['SentimentScores'])
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[-1]['MessageText']

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


'❗️🇷🇺🇺🇸 #рынки #ожидания #россия #сша  Нефтегаз🛢 Газпром (GAZP) Роснефть (ROSN) ГазпромНефть (SIBN) Новатэк (NVTK) Сургутнефтегаз (SNGSP) Лукойл (LKOH) Татнефть (TATN) Башнефть (BANEP) Транснефть (TRNFP) Саратовский НПЗ (KRKNP)  Финансы💵 Сбербанк (SBER) Банк Санкт-Петербург (BSPB) ВТБ (VTBR) Тинькофф (TCSG) Мосбиржа (MOEX) СПБ Биржа (SPBE) QIWI (QIWI) Ренессанс страхование (RENI) АФК СИСТЕМА (AFKS) SFI (SFIN)  ИТ🌐 Yandex (YNDX) Positive Technologies (POSI) HeadHunter (HHR) VK/MAIL (VKCO) Softline (SFTL)  Удобрения и нефтехимия 🧪 Акрон (AKRN) ФосАгро (PHOR) КуйбышевАзот (KAZT) Нижнекамскнефтехим (NKNCP) Казаньоргсинтез (KZOS) Нижнекамскшина (NKSH)  Ритейл 🛒 OZON (OZON) Магнит (MGNT) Fix price (FIXP) Beluga Group(BELU) Абрау Дюрсо (ABRD)  Детский Мир (DSKY)  Лента (LNTA) М.Видео-Эльдорадо (MVID) Окей (OKEY) X5 Group (FIVE) Обувь России (ORUP)  Энергетика ⚡️ ИнтерРАО (IRAO) Россети Центр (MRKC) Россети Центра и Приволжья (MRKP) Юнипро (UPRO) ОГК-2 (OGKB) ТГК-1 (TGKA)  Русгидро (HYDR) Энел

In [55]:
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]

MessageText               🇷🇺#ABIO  После интервью Марии Воронцовой о раз...
issuerid                                                              [266]
SentimentScores                                                         [4]
n_companies_in_message                                                    1
Name: 4662, dtype: object

In [56]:
messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['issuerid']

[266]

In [57]:
#!python -m spacy download ru_core_news_sm
import spacy

In [58]:
text = messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['MessageText']
entities_id = messages_sentiments_company.sort_values(by='n_companies_in_message').iloc[1]['issuerid']

clean_text = re.sub(r'[^\w\s]', '', text)

nlp = spacy.load("ru_core_news_sm")

tokenizer = nlp.tokenizer

text = clean_text
tokens = tokenizer(text)

print(text)

for entity_id in entities_id:
    for entity_name in dict_company_names[entity_id]:
        window_words = len(tokenizer(entity_name))
        print(entity_name, window_words)
        for i in range(len(tokens) - window_words + 1):
            token_texts = [token.text for token in tokens[i:i+window_words]]
            levenshtein_ratio = Levenshtein.ratio(''.join(entity_name), ''.join(token_texts))
            print(levenshtein_ratio)




ABIO  После интервью Марии Воронцовой о развитии биотехнологий в России в секторе генетики началась охота за активами РФарм Протек и ФармаИмпекс интересуются покупкой лидера отрасли компании Артген Биотех  BRIEF
Artgen biotech 2
0.21052631578947367
0.09999999999999998
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.09523809523809523
0.09999999999999998
ABIO 1
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.4444444444444444
Артген Биотех 2
0.11111111111111116
0.3157894736842105
0.3076923076923077
0.23076923076923073
0.2142857142857143
0.25
0.2727272727272727
0.4117647058823529
0.37037037037037035
0.09999999999999998
0.09999999999999998
0.2857142857142857
0.3571428571428571
0.27586206896551724
0.23076923076923073
0.19999999999999996
0.17391304347826086
0.15384615384615385
0.33333333333333337
0.4
0.16000000000000003
0.2777777777777778
0.18181818181818177
0.14

In [59]:
import re

text = dfmentions['MessageText'][2].lower()

for substring in company_names_lower:
    matches = re.finditer(substring, text)
    
    for match in matches:
        start_position = match.start()
        end_position = match.end()
        print(substring, (start_position, end_position, "ORG"))


зил (47, 50, 'ORG')
северсталь (6, 16, 'ORG')


In [60]:
import spacy
import re

# Загрузка модели SpaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Преобразование текста в нижний регистр
text = dfmentions['MessageText'][2].lower()

# Найти совпадения для каждого названия компании
for company_name in company_names_lower:
    # Найти совпадения с использованием регулярного выражения
    matches = re.finditer(company_name, text)
    
    # Для каждого совпадения
    for match in matches:
        start_position = match.start()
        end_position = match.end()
        
        # Токенизация с использованием SpaCy
        doc = nlp(text)
        for token in doc:
            # Проверяем, в каких токенах находится совпадение
            if token.idx <= start_position < token.idx + len(token.text):
                start_token = token
            if token.idx <= end_position < token.idx + len(token.text):
                end_token = token
                
        # Выводим совпадение
        print(company_name, (start_token.text, end_token.text, "ORG"))


зил ('бразилию', 'бразилию', 'ORG')
северсталь ('северсталь', 'бразилию', 'ORG')


In [44]:
#training_data

NameError: name 'training_data' is not defined

In [62]:
ner = nlp.get_pipe("ner")


In [63]:
import spacy
from spacy.training.example import Example

# Загрузка модели SpaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Добавление компаний в модель SpaCy

for company_name in company_names:
    ner.add_label(company_name)

# Пример обучающего набора данных
training_data = [
    ("Элон Маск основал SpaceX и Tesla.", {"entities": [(0, 9, "PERSON"), (20, 26, "SpaceX"), (29, 34, "Tesla")]}),

    # Добавьте больше предложений и соответствующих меток
]

# Обучение модели
ner = nlp.get_pipe("ner")
for _, annotations in training_data:
    example = Example.from_dict(nlp.make_doc(_), annotations)
    nlp.update([example])

# Пример тестовых данных
test_sentences = [
    "Сотрудники SpaceX рады новому проекту.",
    "Сотрудники Роснефть рады новому проекту.",
    "Роснефтью никто не торгует!",
    "Акционерная компания 'АЛРОСА' (публичное акционерное общество)",
    "Роснефть (ROSN) справедливая цена 425.17 рубля, потенциал роста на 23%. #сборник  1. Роснефть увеличила добычу на 3% в 2022 году, планирует увеличить ещё на 5% в 2023 https://t.me/AK47pfl/14309 2. Саудовская Аравия повышает цены реализации по нефти для всех регионов https://t.me/AK47pfl/14311 3. Нефть и газ зарабатывают много денег: BP решили увеличить инвестиции в добычу нефти https://t.me/AK47pfl/14313 4. Российская нефть проходит ограничения лучше прогнозов https://t.me/AK47pfl/14303 5. Цены реализации российской нефти, похоже, выше, чем принято считать https://t.me/AK47pfl/14304  @AK47pfl",
    "Кто торгует Норникелем?"
    
    # Добавьте больше предложений для тестирования
]

# Предсказание сущностей в тестовых данных
for sentence in test_sentences:
    doc = nlp(sentence)
    print("Текст:", sentence)
    for ent in doc.ents:
        print("Сущность:", ent.text, "-", ent.label_)


Текст: Сотрудники SpaceX рады новому проекту.
Сущность: SpaceX - ORG
Текст: Сотрудники Роснефть рады новому проекту.
Сущность: Роснефть - ORG
Текст: Роснефтью никто не торгует!
Сущность: Роснефтью - ORG
Текст: Акционерная компания 'АЛРОСА' (публичное акционерное общество)
Сущность: 'АЛРОСА' - ORG
Текст: Роснефть (ROSN) справедливая цена 425.17 рубля, потенциал роста на 23%. #сборник  1. Роснефть увеличила добычу на 3% в 2022 году, планирует увеличить ещё на 5% в 2023 https://t.me/AK47pfl/14309 2. Саудовская Аравия повышает цены реализации по нефти для всех регионов https://t.me/AK47pfl/14311 3. Нефть и газ зарабатывают много денег: BP решили увеличить инвестиции в добычу нефти https://t.me/AK47pfl/14313 4. Российская нефть проходит ограничения лучше прогнозов https://t.me/AK47pfl/14303 5. Цены реализации российской нефти, похоже, выше, чем принято считать https://t.me/AK47pfl/14304  @AK47pfl
Сущность: Роснефть - ORG
Сущность: Саудовская Аравия - LOC
Сущность: BP - ORG
Текст: Кто торгуе

c:\Users\Глеб\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Элон Маск основал SpaceX и Tesla." with entities "[(0, 9, 'PERSON'), (20, 26, 'SpaceX'), (29, 34, 'T...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


In [64]:
spacy.tokenizer()

TypeError: 'module' object is not callable

In [67]:
import spacy
from spacy.tokens import Span
from spacy.training import Example

# Загрузка модели SpaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Список названий компаний и их псевдонимов
company_aliases = [
    ("ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО \"БАНК \"САНКТ-ПЕТЕРБУРГ\"", ["BSPB RX", "Банк Санкт-Петербург", "БСПБ", "Банк Петербург", "СПБ Банк"]),
    ("Публичное акционерное общество \"БАНК УРАЛСИБ\"", ["USBN"]),
    ("Публичное акционерное общество \"Башинформсвязь\"", ["BISV"]),
    ("Публичное акционерное общество \"Белуга Групп\"", ["BELU RX", "Белуга"]),
    ("Публичное акционерное общество \"Бест Эффортс Банк\"", ["ALBK"]),
    ("Публичное акционерное общество \"Бурятзолото\"", ["BRZL"]),
    ("Публичное акционерное общество \"Варьеганнефтегаз\"", ["VJGZ"]),
    ("Публичное акционерное общество \"Владимирский химический завод\"", ["VLHZ"]),
    ("Публичное акционерное общество \"Волгоградэнергосбыт\"", ["VGSB"]),
    ("Публичное акционерное общество \"Вторая генерирующая компания оптового рынка электроэнергии\"", ["OGKB"]),
    ("Публичное акционерное общество \"Выборгский судостроительный завод\"", ["VSYD"]),
    ("Публичное акционерное общество \"ГАЗ\"", ["GAZA", "группа \"ГАЗ\"", "группа ГАЗ", "ПАО Газ", "ПАО \"Газ\""]),
    ("Публичное акционерное общество \"ГАЗКОН\"", ["GAZC", "газкон", "газкон-ао"]),
    ("Публичное акционерное общество \"Газпром газораспределение Ростов-на-Дону\"", ["RTGZ", "ПАО \"Газпром газораспределение Ростов-на-Дону\"", "rostovoblgaz", "Газпром РнД", "Ростовоблгаз"]),
    ("Публичное акционерное общество \"Газпром нефть\"", ["SIBN RX", "ПАО \"Газпром нефть\"", "Газпром нефть", "Gazprom Neft"]),
    ("Публичное акционерное общество \"Газпром\"", ["GAZP RX", "ПАО \"Газпром\"", "Газпром", "Gazprom", "gazp"]),
]

# Функция для добавления псевдонимов компаний
def add_company_alias(doc):
    for company, aliases in company_aliases:
        for alias in aliases:
            if alias in doc.text:
                alias_span = doc.text.find(alias)
                doc.ents = list(doc.ents) + [Span(doc, alias_span, alias_span + len(alias), label="COMPANY")]
    return doc

# Добавление пользовательской разделительной метки для компаний
nlp.add_pipe(add_company_alias, after="ner")

# Объединение сущностей и существительных
nlp.add_pipe("merge_entities")  # Объединение сущностей
nlp.add_pipe("merge_noun_chunks")  # Объединение существительных

# Пример текста для обработки
text = "Сотрудники SpaceX рады новому проекту. Мы также работаем с ПУБЛИЧНЫМ АКЦИОНЕРНЫМ ОБЩЕСТВОМ 'БАНК 'САНКТ-ПЕТЕРБУРГ'."

# Обработка текста
doc = nlp(text)

# Вывод результатов
print("Сущности в тексте:")
for ent in doc.ents:
    print(ent.text, "-", ent.label_)


ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <function add_company_alias at 0x00000156E2D1BEC0> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.

In [66]:
import spacy
from spacy.tokens import Span
from spacy.language import Language

# Загрузка модели SpaCy для русского языка
nlp = spacy.load("ru_core_news_sm")

# Список названий компаний и их псевдонимов
company_aliases = [
    ("ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО \"БАНК \"САНКТ-ПЕТЕРБУРГ\"", ["BSPB RX", "Банк Санкт-Петербург", "БСПБ", "Банк Петербург", "СПБ Банк"]),
    ("Публичное акционерное общество \"БАНК УРАЛСИБ\"", ["USBN"]),
    ("Публичное акционерное общество \"Башинформсвязь\"", ["BISV"]),
    ("Публичное акционерное общество \"Белуга Групп\"", ["BELU RX", "Белуга"]),
    ("Публичное акционерное общество \"Бест Эффортс Банк\"", ["ALBK"]),
    ("Публичное акционерное общество \"Бурятзолото\"", ["BRZL"]),
    ("Публичное акционерное общество \"Варьеганнефтегаз\"", ["VJGZ"]),
    ("Публичное акционерное общество \"Владимирский химический завод\"", ["VLHZ"]),
    ("Публичное акционерное общество \"Волгоградэнергосбыт\"", ["VGSB"]),
    ("Публичное акционерное общество \"Вторая генерирующая компания оптового рынка электроэнергии\"", ["OGKB"]),
    ("Публичное акционерное общество \"Выборгский судостроительный завод\"", ["VSYD"]),
    ("Публичное акционерное общество \"ГАЗ\"", ["GAZA", "группа \"ГАЗ\"", "группа ГАЗ", "ПАО Газ", "ПАО \"Газ\""]),
    ("Публичное акционерное общество \"ГАЗКОН\"", ["GAZC", "газкон", "газкон-ао"]),
    ("Публичное акционерное общество \"Газпром газораспределение Ростов-на-Дону\"", ["RTGZ", "ПАО \"Газпром газораспределение Ростов-на-Дону\"", "rostovoblgaz", "Газпром РнД", "Ростовоблгаз"]),
    ("Публичное акционерное общество \"Газпром нефть\"", ["SIBN RX", "ПАО \"Газпром нефть\"", "Газпром нефть", "Gazprom Neft"]),
    ("Публичное акционерное общество \"Газпром\"", ["GAZP RX", "ПАО \"Газпром\"", "Газпром", "Gazprom", "gazp"]),
]

# Класс для добавления псевдонимов компаний в качестве сущностей
class CompanyAliasComponent:
    def __init__(self, nlp):
        self.company_aliases = company_aliases
        self.labels = set()
        for _, aliases in self.company_aliases:
            self.labels.update(aliases)
        Span.set_extension("is_company_alias", default=False)

    def __call__(self, doc):
        for company, aliases in self.company_aliases:
            for alias in aliases:
                if alias in doc.text:
                    alias_span = doc.text.find(alias)
                    alias_ent = Span(doc, alias_span, alias_span + len(alias), label="COMPANY_ALIAS")
                    alias_ent._.is_company_alias = True
                    doc.ents = list(doc.ents) + [alias_ent]
        return doc

@Language.component("company_alias_component")
def create_company_alias_component(doc):
    return CompanyAliasComponent(nlp)(doc)

# Добавление компонента для обработки псевдонимов компаний в конвейер обработки текста
nlp.add_pipe("company_alias_component")

# Обработка текста
text = "Сотрудники SpaceX рады новому проекту. Мы также работаем с ПУБЛИЧНЫМ АКЦИОНЕРНЫМ ОБЩЕСТВОМ 'БАНК 'САНКТ-ПЕТЕРБУРГ'."
doc = nlp(text)

# Вывод результатов
print("Сущности в тексте:")
for ent in doc.ents:
    print(ent.text, "-", ent.label_)


Сущности в тексте:
SpaceX - ORG
ПУБЛИЧНЫМ - ORG
АКЦИОНЕРНЫМ ОБЩЕСТВОМ 'БАНК 'САНКТ-ПЕТЕРБУРГ' - ORG


In [ ]:
#Что делать с 0 в выборке?
#В случае если в тексте нет информации о кампании выводить пустой []
#
#